# PyRosettaCluster 
## Tutorial 1B. Reproduce simple protocol

PyRosettaCluster Tutorial 1B uses the `pyrosetta.distributed.cluster` python module to reproduce a decoy generated by a PyRosetta simulation previosly run in PyRosettaCluster Tutorial 1A, using only an input `.pdb` file and the original user-provided PyRosetta protocol(s).

In PyRosettaCluster Tutorial 1A, you used `PyRosettaCluster` to apply a PyRosetta protocol to an input `.pdb` file, and generated several output `.pdb` files. Each output `.pdb` file contains information needed to exactly reproduce it.

*Note:* This Jupyter notebook uses parallelization and is **not** meant to be executed within a Google Colab environment.

*Note:* This Jupyter notebook requires the PyRosetta distributed layer which is obtained by building PyRosetta with the `--serialization` flag or installing PyRosetta from the RosettaCommons conda channel 

**Please see Chapter 16.00 for setup instructions**

*Note:* This Jupyter notebook is intended to be run within **Jupyter Lab**, but may still be run as a standalone Jupyter notebook.

### 1. Import packages

In [1]:
import bz2
import json
import glob
import logging
import os
import pandas as pd
import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.viewer as viewer

from pyrosettacluster import PyRosettaCluster, reproduce

logging.basicConfig(level=logging.INFO)

### 2. Initialize a compute cluster using `dask` 

See Tutorial 1A to review:
1. Click the "Dask" tab in Jupyter Lab <i>(arrow, left)</i>
2. Click the "+ NEW" button to launch a new compute cluster <i>(arrow, lower)</i>

3. Once the cluster has started, click the brackets to "inject client code" for the cluster into your notebook

Inject client code here, then run the cell:

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:40329")
client

Client Scheduler: tcp://127.0.0.1:40329 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.63 GB


### 3. Re-define or import the original user-provided PyRosetta protocol:

The purpose of the `sha1` attribute of `PyRosettaCluster` is to ensures that you have committed all of your untracked changes into your git repository before executing the original simulation. When you run the `reproduce` function, the original `sha1` attribute of `PyRosettaCluster` was captured in the output decoy `.pdb` file which ensures that you have checked out the same git SHA1 hash before reproducing the simulation. In this way, `my_protocol` remains statically captured at the git SHA1 hash from the original simulation. However, you may always update `my_protocol`, commit your changes to your git repository, and re-run the simulation, because the `sha1` attribute of `PyRosettaCluster` automatically detects the new git SHA1 hash in your git repository.

In [3]:
from my_protocols import my_protocol
client.upload_file("my_protocols.py") # This sends a local file up to all worker nodes.

### 4. Reproduce the original decoy:

The simulation in Tutorial 1A generated four decoys (because `nstruct=4` in the original simulation). Let's say we'd like to reproduce the decoy with the lowest energy. First, let's inspect the results with the `pandas` library:

In [4]:
original_results = glob.glob(os.path.join(os.getcwd(), "outputs_1A", "decoys", "*", "*.pdb.bz2"))

data = {}
for original_result in original_results:
    with open(original_result, "rb") as f:
        pdbstring = bz2.decompress(f.read()).decode()
        for line in reversed(pdbstring.split("\n")):
            remark = "REMARK PyRosettaCluster: "
            if line.startswith(remark):
                data[original_result] = json.loads(line.split(remark)[-1])["scores"]
                break

df = pd.DataFrame().from_records(data).T
df

,dslf_fa13,fa_atr,fa_dun,fa_elec,fa_intra_rep,fa_intra_sol_xover4,fa_rep,fa_sol,hbond_bb_sc,hbond_lr_bb,hbond_sc,hbond_sr_bb,lk_ball_wtd,omega,p_aa_pp,pro_close,rama_prepro,ref,total_score,yhh_planarity
/shared/home/jklima/git/PyRosettaCluster/tutorials/outputs_1A/decoys/0000/2020.05.30.20.03.19.467398_18806b9f61aa455cb6b35383beaba2f0.pdb.bz2,0.0,-518.611945,151.917093,-176.795881,216.690803,19.277644,110.812163,316.594177,-8.654614,-40.303857,-23.831458,-37.080229,-7.157926,15.202165,-36.681278,0.0,-18.494006,24.51825,-307.588756,0.536283
/shared/home/jklima/git/PyRosettaCluster/tutorials/outputs_1A/decoys/0000/2020.05.30.20.03.19.467398_89381c4208904a9aaf5e21c669174739.pdb.bz2,0.0,-511.028454,156.044952,-180.607435,216.573583,18.456979,108.788886,311.903317,-10.415312,-40.154626,-22.948554,-36.627020,-6.680779,14.324375,-35.693698,0.0,-21.097263,24.51825,-307.924130,1.106433
/shared/home/jklima/git/PyRosettaCluster/tutorials/outputs_1A/decoys/0000/2020.05.30.20.03.19.467398_bce36646fa7748bd88059c5a5032c094.pdb.bz2,0.0,-512.449228,159.024131,-174.329291,213.182679,19.069250,117.069985,301.465705,-8.494807,-40.636754,-25.328228,-33.640772,-7.594040,14.667233,-37.022140,0.0,-17.875183,24.51825,-304.710835,0.524812
/shared/home/jklima/git/PyRosettaCluster/tutorials/outputs_1A/decoys/0000/2020.05.30.20.03.19.467398_fcf9ac5dc7794e6e99859e936f94c35e.pdb.bz2,0.0,-513.090895,153.255386,-180.102696,211.873175,18.256771,106.703676,313.867180,-9.507289,-40.466284,-23.974687,-36.657563,-6.597737,17.066356,-35.729644,0.0,-16.621724,24.51825,-308.540925,0.447818


Now locate the decoy with the lowest Rosetta `total_score` to reproduce:

In [5]:
decoy_to_reproduce = df.sort_values(by="total_score", ascending=True).index[0]
decoy_to_reproduce

'/shared/home/jklima/git/PyRosettaCluster/tutorials/outputs_1A/decoys/0000/2020.05.30.20.03.19.467398_fcf9ac5dc7794e6e99859e936f94c35e.pdb.bz2'

### 5. Launch the reproduction simulation using `reproduce()`:

Reproducing the decoy is accomplished with the `reproduce()` function of the `pyrosettacluster` module. This method requires the `.pdb` or `.pdb.bz2` file to reproduce: `input_file`. Alternatively, a `scorefile` with full simulation records and a `decoy_name` may be provided to `reproduce()` instead of the `.pdb` or `.pdb.bz2` file. The user-provided PyRosetta protocol(s) must be defined or imported and input into `reproduce()` as the `protocols` argument parameter. The user is responsible for supplying the same protocol that was used in the original simulation! Additionally, any supplied `instance_kwargs` will override any `PyRosettaCluster` instance attributes from the `input_file` or `scorefile`. This may be useful when, for example, you want to change your cluster configuration while reproducing a decoy.

In [6]:
output_path = os.path.join(os.getcwd(), "outputs_1B")

reproduce(
    input_file=decoy_to_reproduce,
    input_packed_pose=None, # Optional, if you used the `input_packed_pose` attribute of `PyRosettaCluster` in the original simulation
    client=client, # Optional
    instance_kwargs={"output_path": output_path, "nstruct": 1}, # Specify new output path, and set `nstruct` to 1 to reproduce the decoy only once. 
    protocols=[my_protocol],
)

INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /shared/home/jklima/.conda/envs/jupyterlab/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.15+release.3121c734db02d2b62dd1974dcb8daface3f50057 2020-04-10T09:29:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


### 6. Visualize the reproduced decoy:

In [8]:
reproduced_results = glob.glob(os.path.join(output_path, "decoys", "*", "*.pdb.bz2"))
assert len(reproduced_results) == 1
with open(reproduced_results[0], "rb") as f:
    reproduced_packed_pose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())

In [9]:
view = viewer.init(reproduced_packed_pose, window_size=(800, 600))
view.add(viewer.setStyle())
view.add(viewer.setStyle(colorscheme="whiteCarbon", radius=0.25))
view.add(viewer.setHydrogenBonds())
view.add(viewer.setHydrogens(polar_only=True))
view.add(viewer.setDisulfides(radius=0.25))
view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### 7. Optionally, perform sanity checks to confirm that the reproduced decoy is identical to the original decoy:

PyRosetta trajectories are _deterministic_ depending on the input random number generated seed(s)!

In [10]:
with open(decoy_to_reproduce, "rb") as f:
    original_packed_pose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
original_pose = original_packed_pose.pose
reproduced_pose = reproduced_packed_pose.pose

#### Assert that the sequences are identical:

In [11]:
assert original_pose.sequence() == reproduced_pose.sequence()

#### Assert that the `total_score`s are identical:

In [12]:
scorefxn = pyrosetta.create_score_function("ref2015.wts")
assert scorefxn(original_pose) == scorefxn(reproduced_pose)

#### Assert that the C$_{\alpha}$–C$_{\alpha}$ root-mean-square deviation (RMSD) is `0.0` Å:

Note: There is no need to first superimpose the `original_pose` and `reproduced_pose` because they were both generated starting from the same `input_packed_pose`

In [14]:
assert pyrosetta.rosetta.core.scoring.CA_rmsd(original_pose, reproduced_pose) == 0.0

### Congrats! 
You have successfully reproduced a PyRosetta simulation using the `pyrosettacluster` module!